# 1. Import and Install Dependencies

In [3]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install opencv-python 
!pip install mediapipe 
!pip install sklearn 
!pip install matplotlib

  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached wrapt-1.14.1-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl (1.5 MB)
  Using cached grpcio-1.59.3-cp311-

  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [44 lines of output]
  Traceback (most recent call last):
    File "C:\Users\maisa\anaconda3\envs\notebook\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
      parsed = _parse_requirement(requirement_string)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\maisa\anaconda3\envs\notebook\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\maisa\anaconda3\envs\notebook\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\maisa

  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
  Using cached mediapipe-0.10.8-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached matplotlib-3.8.2-cp311-cp311-win_amd64.whl.metadata (5.9 kB)
  Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.4.6-py3-none-win_amd64.whl (199 kB)
  Using cached contourpy-1.2.0-cp311-cp311-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/158.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/158.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/158.4 kB ? eta -:--:--
     ------- ----------------------------- 30.7/158.4 kB 187.9 kB/s eta 0:00:01
     --------- --------------------------- 41.0/158.4 kB 219.4 kB/s eta 0:00:01
     ---------------- -----

In [1]:
import cv2
import numpy as np
import os 
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [7]:
def mediapipe_detection(image,model):
    image =cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results


In [8]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)    

In [9]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,70,10),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                             )    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)
                             )    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                             )    

In [97]:
pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [99]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose,face,lh,rh])

# 4. Setup folders for data collection

In [13]:
DATA_PATH=os.path.join('MP_Data')
actions = np.array(['pain','feaver','doctor','injury','yestrday','tomorow','injection','dizziness','allergy','bandage'])
no_sequences=30
sequence_length=30


In [14]:
# for action in actions :
#     for sequence in range(no_sequences):
#         try:
#             os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
#         except:
#             pass

# 5. Collect keypoints

In [15]:
# cap=cv2.VideoCapture(0)
 
# with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
#     for action in actions:
#         for sequence in range(no_sequences):
#             for frame_num in range(sequence_length):
#                 ret,frame=cap.read()
#                 image,results=mediapipe_detection(frame,holistic)
#                 #print(results)
#                 draw_styled_landmarks(image,results)
                
#                 if frame_num ==0:
#                     cv2.putText(image,"Starting Collection ",(120,200),
#                                cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
#                     cv2.putText(image,f"Collecting frames for {action} video number {sequence} ",(15,12),
#                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(500)
#                 else:
#                     cv2.putText(image,f"Collecting frames for {action} video number {sequence} ",(15,12),
#                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)
                    
#                 keypoints=extract_keypoints(results)
#                 npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
#                 np.save(npy_path,keypoints)
#                 #cv2.imshow('OpenCV Feed',image)

#                 if cv2.waitKey(10) & 0xff==ord('q'):
#                     break
#     cap.release()
#     cv2.destroyAllWindows()

# 6. Preprocess Data and Create labels and features

In [16]:
!pip install scikit-learn

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [18]:
label_map={label:num for num,label in enumerate(actions)}

In [19]:
label_map

{'pain': 0,
 'feaver': 1,
 'doctor': 2,
 'injury': 3,
 'yestrday': 4,
 'tomorow': 5,
 'injection': 6,
 'dizziness': 7,
 'allergy': 8,
 'bandage': 9}

In [20]:
sequences ,labels =[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),f'{frame_num}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [21]:
len(labels)

300

In [22]:
X=np.array(sequences)

In [23]:
X.shape

(300, 30, 1662)

In [24]:
y= to_categorical(labels).astype(int)

In [25]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [27]:
y_test.shape

(15, 10)

# 7. Build and train LSTM Neural Network

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense ,GRU
from tensorflow.keras.callbacks import TensorBoard

In [29]:
log_dir=os.path.join("Logs")
tb_callback=TensorBoard(log_dir=log_dir)

In [89]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [31]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["categorical_accuracy"])

In [33]:
model.fit(X_train,y_train,epochs=180,callbacks=[tb_callback])

Epoch 1/180
9/9 [==============================] - 12s 126ms/step - loss: 3.1490 - categorical_accuracy: 0.1088
Epoch 2/180
9/9 [==============================] - 1s 116ms/step - loss: 2.3414 - categorical_accuracy: 0.0842
Epoch 3/180
9/9 [==============================] - 1s 138ms/step - loss: 2.2482 - categorical_accuracy: 0.1158
Epoch 4/180
9/9 [==============================] - 1s 134ms/step - loss: 2.5139 - categorical_accuracy: 0.1053
Epoch 5/180
9/9 [==============================] - 1s 132ms/step - loss: 2.3017 - categorical_accuracy: 0.1123
Epoch 6/180
9/9 [==============================] - 1s 127ms/step - loss: 2.3000 - categorical_accuracy: 0.1193
Epoch 7/180
9/9 [==============================] - 1s 141ms/step - loss: 2.2979 - categorical_accuracy: 0.1123
Epoch 8/180
9/9 [==============================] - 1s 130ms/step - loss: 2.3046 - categorical_accuracy: 0.1228
Epoch 9/180
9/9 [==============================] - 1s 119ms/step - loss: 2.2736 - categorical_accuracy: 0.1123


In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 596906 (2.28 MB)
Trainable params: 596906 

# 8. model prediction

In [36]:
res=model.predict(X_test)


1/1 [==============================] - 1s 1s/step


In [34]:
# Assuming you have predicted labels in 'y_pred' and true labels in 'y_true'
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

NameError: name 'confusion_matrix' is not defined

In [59]:
actions[np.argmax(res[4])]


'dizziness'

In [37]:
actions[np.argmax(y_test[1])]

'injury'

# 9. Save Weights

In [38]:
model.save('actionv3.h5')

C:\Users\maisa\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 10.Evaluation using Confusion Matrix and Accuracy

In [39]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [40]:
yhat=model.predict(X_test)

1/1 [==============================] - 0s 75ms/step


In [41]:
ytrue=np.argmax(y_test,axis=1).tolist()
yhat=np.argmax(yhat,axis=1).tolist()

In [42]:
ytrue

[1, 3, 4, 7, 8, 2, 0, 5, 7, 3, 6, 9, 4, 7, 6]

In [43]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[14,  0],
        [ 1,  0]],

       [[14,  0],
        [ 1,  0]],

       [[12,  2],
        [ 0,  1]],

       [[11,  2],
        [ 2,  0]],

       [[13,  0],
        [ 1,  1]],

       [[13,  1],
        [ 1,  0]],

       [[13,  0],
        [ 1,  1]],

       [[10,  2],
        [ 0,  3]],

       [[14,  0],
        [ 0,  1]],

       [[14,  0],
        [ 0,  1]]], dtype=int64)

In [44]:
accuracy_score(ytrue,yhat)

0.5333333333333333

# 11. Test in Real time

In [53]:
!pip install gtts

  Obtaining dependency information for gtts from https://files.pythonhosted.org/packages/a7/ef/190f64a4edeb13165e3c598a08f06a2ae80cdae0aa208c96c20efdb7ad4b/gTTS-2.4.0-py3-none-any.whl.metadata


In [82]:
model2 = Sequential()
model2.add(GRU(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model2.add(GRU(128, return_sequences=True, activation='relu'))
model2.add(GRU(64, return_sequences=False, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(actions.shape[0], activation='softmax'))

In [90]:
model.load_weights('actionv2.h5')

In [91]:
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
          (255, 0, 255), (128, 128, 0), (0, 128, 128), (128, 0, 128), (0, 128, 0)]
def prob_viz(res,actions,input_frame,colors):
    output_frame=input_frame.copy()
    for num,prob in enumerate(res):
        cv2.rectangle(output_frame,(0,60+num*40),(int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

In [98]:
sequence=[]
sentence=[]
predictions=[]
threshold=0.7

cap=cv2.VideoCapture(0)
 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image,results=mediapipe_detection(frame,holistic)
        #print(results)
        draw_styled_landmarks(image,results)
        
        keypoints=extract_keypoints(results)
        sequence.append(keypoints)
        sequence=sequence[-30:]
        
        if len(sequence)==30:
            res=model2.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)]>threshold:
                    if len(sentence)>0:
                        if actions[np.argmax(res)]!=sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else: 
                        sentence.append(actions[np.argmax(res)])

            if len(sentence)>5:
                sentence=sentence[-5:]
            #image= prob_viz(res,actions,image,colors)
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

InvalidArgumentError: Graph execution error:

Detected at node sequential_3/gru/while/gru_cell/BiasAdd defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\maisa\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\maisa\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\maisa\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\maisa\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\maisa\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\maisa\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\maisa\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\maisa\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 516, in dispa

In [100]:
cap.release()
cv2.destroyAllWindows()

In [103]:
 from gtts import gTTS
import os

sequence=[]
sentence=[]
predictions=[]
threshold=0.7

cap=cv2.VideoCapture(0)
 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image,results=mediapipe_detection(frame,holistic)
        #print(results)
        draw_styled_landmarks(image,results)
        
        keypoints=extract_keypoints(results)
        sequence.append(keypoints)
        sequence=sequence[-30:]
        
        if len(sequence)==30:
            res=model2.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)]>threshold:
                    if len(sentence)>0:
                        if actions[np.argmax(res)]!=sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else: 
                        sentence.append(actions[np.argmax(res)])

            if len(sentence)>5:
                sentence=sentence[-5:]
            #image= prob_viz(res,actions,image,colors)
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    #Text you want to convert to speech
    text_to_speak = sentence

    # Language in which you want to convert (optional, default is 'en' - English)
    language = 'en'

    # Create a gTTS object
    if text_to_speak == sentence:
        tts = gTTS(text=text_to_speak, lang=language, slow=False)
        # Save the speech as an audio file
        tts.save("output.mp3")
        # Play the generated audio file (requires a media player)
        os.system("start output.mp3")

1/1 [==============================] - 0s 50ms/step
feaver
1/1 [==============================] - 0s 24ms/step
feaver
1/1 [==============================] - 0s 24ms/step
feaver
1/1 [==============================] - 0s 26ms/step
feaver
1/1 [==============================] - 0s 22ms/step
feaver
1/1 [==============================] - 0s 25ms/step
feaver
1/1 [==============================] - 0s 25ms/step
feaver
1/1 [==============================] - 0s 23ms/step
feaver
1/1 [==============================] - 0s 22ms/step
feaver
1/1 [==============================] - 0s 28ms/step
feaver
1/1 [==============================] - 0s 25ms/step
feaver
1/1 [==============================] - 0s 27ms/step
feaver
1/1 [==============================] - 0s 29ms/step
feaver
1/1 [==============================] - 0s 26ms/step
feaver
1/1 [==============================] - 0s 29ms/step
feaver
1/1 [==============================] - 0s 21ms/step
feaver
1/1 [==============================] - 0s 24ms/step
feav

TypeError: 'fp' is not a file-like object or it does not take bytes: 'list' object has no attribute 'strip'